In [750]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from datetime import datetime

In [519]:
test = pd.read_csv('test.csv')

test.sample(3)

In [520]:
mar14 = pd.read_csv('auto_ru_14mar2021_full.csv')

# обработка для объединения баз

In [521]:
mar14 = mar14[mar14.bodyType.notna()].reset_index(drop=True)

In [522]:
# bodyType
def bt(x):
    for i in bodytypes:
        if i in x.lower(): return i

mar14.bodyType = mar14.bodyType.apply(bt) # приводим 

In [523]:
# делаем нормальный один столбец с цветом
clr = {x: 0 for x in mar14['color_hex'].unique()}

for x in clr: 
    try:
        clr[x] = mar14[mar14['color_hex'] == x]['color'].value_counts().index[0]
    except:
        clr[x] = 'away'

clr['FFC0CB'], clr['FFD600'], clr['040001'] = 'розовый', 'жёлтый', 'черный'

mar14['color_hex'] = mar14['color_hex'].apply(lambda x: clr[x])
mar14['color'] = mar14['color_hex']
mar14 = mar14.drop(['color_hex', 'color2'], axis=1)

In [524]:
# удалим строки в которых нет цены
mar14 = mar14.dropna(subset = ['Price'], axis=0).reset_index(drop=True)

In [525]:
mar14 = mar14.drop(['Unnamed: 0'], axis=1)

In [526]:
# приводим к нужному виду колонку engineDisplacement и удаляем электрокары

def engineDisplacement_func(x):
    try:
        return re.findall(r'\d\.\d', x)[0] + ' LTR'
    except:
        return ' LTR'

mar14['engineDisplacement'] = mar14['engineDisplacement'].apply(engineDisplacement_func)

In [527]:
# приводим к одному виду колонку enginePower

mar14['enginePower'] = mar14['enginePower'].apply(lambda x: str(int(x)) + ' N12')

In [528]:
# приводим в порядок колонку fuelType

fuelType_dict = {'GASOLINE': 'бензин', 'DIESEL': 'дизель', 'ELECTRO': 'электро', 
                 'HYBRID': 'гибрид', 'LPG': 'газ'}

mar14['fuelType'] = mar14['fuelType'].apply(lambda x: fuelType_dict[x])

In [529]:
# приводим в порядок колонку mileage

mar14['mileage'] = mar14['mileage'].apply(lambda x: int(x))

In [530]:
# приводим в порядок колонку modelDate

mar14['modelDate'] = mar14['modelDate'].apply(lambda x: int(x))

In [531]:
# приводим в порядок колонку numberOfDoors

mar14['numberOfDoors'] = mar14['numberOfDoors'].apply(lambda x: np.int64(x))

In [532]:
# приводим в порядок колонку parsing_unixtime

mar14['parsing_unixtime'] = mar14['parsing_unixtime'].apply(lambda x: np.int64(str(x)[:10]))

In [533]:
# приводим в порядок колонку productionDate

mar14['productionDate'] = mar14['productionDate'].apply(lambda x: np.int64(x))

In [534]:
vehicleTransmission_dict = {'MECHANICAL': 'механическая', 'AUTOMATIC': 'автоматическая',
                            'ROBOT': 'роботизированная', 'VARIATOR': 'вариатор'}
mar14['vehicleTransmission'] = mar14['vehicleTransmission'].apply(lambda x: vehicleTransmission_dict[x])

In [535]:
# поправим столбец владельцы

owners_dict = {'1': '1\xa0владелец', '3': '3 или более', '2': '2\xa0владельца', '4': '3 или более'}
mar14 = mar14.dropna(subset = ['Владельцы'], axis=0).reset_index(drop=True)

mar14['Владельцы'] = mar14['Владельцы'].apply(lambda x: owners_dict[str(int(x))])

In [536]:
# затащим столбец Владение

def current_own(date):
    try:
        year = int(re.sub(r"{'year': (\d{4}), 'month': (\d+)}", r"\1", date))
        month = int(re.sub(r"{'year': (\d{4}), 'month': (\d+)}", r"\2", date))
        dt_date = datetime(year, month, 1)
        dt_delta = datetime(2021, 3, 14) - dt_date
        delta_year = dt_delta.days // 360
        delta_month = dt_delta.days % 360 // 30
        
        if delta_year == 0:
            if delta_month in [11, 12, 13, 14]:
                name_month = 'месяцев'
            elif str(delta_month)[-1] == '1':
                name_month = 'месяц'
            elif int(str(delta_month)[-1]) in [2, 3, 4]:
                name_month = 'месяца'
            elif int(str(delta_month)[-1]) in [5, 6, 7, 8, 9, 0]:
                name_month = 'месяцев'
            
            return f'{delta_month} {name_month}'
        
        elif delta_month == 0:
            if delta_year in [11, 12, 13, 14]:
                name_year = 'лет'
            elif str(delta_year)[-1] == '1':
                name_year = 'год'
            elif int(str(delta_year)[-1]) in [2, 3, 4]:
                name_year = 'года'
            elif int(str(delta_year)[-1]) in [5, 6, 7, 8, 9, 0]:
                name_year = 'лет'            
            
            return f'{delta_year} {name_year}'
        else:
            if delta_year in [11, 12, 13, 14]:
                name_year = 'лет'
            elif str(delta_year)[-1] == '1':
                name_year = 'год и'
            elif int(str(delta_year)[-1]) in [2, 3, 4]:
                name_year = 'года и'
            elif int(str(delta_year)[-1]) in [5, 6, 7, 8, 9, 0]:
                name_year = 'лет и'
            if delta_month in [11, 12, 13, 14]:
                name_month = 'месяцев'
            elif str(delta_month)[-1] == '1':
                name_month = 'месяц'
            elif int(str(delta_month)[-1]) in [2, 3, 4]:
                name_month = 'месяца'
            elif int(str(delta_month)[-1]) in [5, 6, 7, 8, 9, 0]:
                name_month = 'месяцев'

            return f'{delta_year} {name_year} {delta_month} {name_month}'
    except:
        return np.nan
    
mar14['Владение'] = mar14['Владение'].apply(current_own)

In [537]:
# тащим колонку ПТС

def PTS(x):
    if x == 'ORIGINAL': return 'Оригинал'
    elif x == 'DUPLICATE': return 'Дубликат'
    else: return np.nan

mar14['ПТС'] = mar14['ПТС'].apply(PTS)

In [538]:
# тащим колонку привод

privod = {'REAR_DRIVE': 'задний', 'FORWARD_CONTROL': 'передний', 'ALL_WHEEL_DRIVE': 'полный'}

mar14['Привод'] = mar14['Привод'].apply(lambda x: privod[x])

In [539]:
# тащим колонку Руль

wheel = {'LEFT': 'Левый', 'RIGHT': 'Правый'}

mar14['Руль'] = mar14['Руль'].apply(lambda x: wheel[x])

In [540]:
mar14.to_csv('autoru_14mar2021_remastered.csv')